In [1]:
import transformers
from transformers import pipeline
import torch

In [2]:
# Check if CUDA is available and set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Function to load the model and tokenizer
def load_model(model_name="google/flan-t5-xl"):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    model = transformers.T5ForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16)
    model.to(device)
    return model, tokenizer

# Function to set up the pipeline
def setup_pipeline(model, tokenizer):
    pipe = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device=0)
    return pipe

# Function to generate text based on prompts
def generate_text(prompt, pipe, max_tokens=100):
    response = pipe(
        prompt, 
        max_length=max_tokens, 
        num_return_sequences=1,
    )
    return response[0]['generated_text']

In [4]:
# Load model and tokenizer
model_name = "google/flan-t5-xl"
model, tokenizer = load_model(model_name)
pipe = setup_pipeline(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"

INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

# Function to read the prompt from a file
def read_prompt_from_file(file_path):
    with open(file_path, 'r') as file:
        prompt = file.read().strip()
    return prompt

# Define the path to the prompt file
prompt_file_path = 'prompt.txt'

# Read the prompt from the file
instruction = read_prompt_from_file(prompt_file_path)

# Format the prompt according to the specified format
formatted_prompt = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)

# Print the formatted prompt to verify
print("Formatted Prompt:\n", formatted_prompt)

# Generate the text
output = generate_text(formatted_prompt, pipe, max_tokens=3000)

# Print the generated text
print("Generated Text:\n", output)

Formatted Prompt:
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
The potential fallacious argument types are:
AA slippery slope
BB ad hominem
CC appeal to (false) authority
DD X appeal to majority
EE No fallacy

Which one of these 5 fallacious argument types does the following text contain?
"There are two distinguished authors on arms control in this country -- there are many others, but two that I want to cite tonight. One is Strobe Talbott in his classic book, "Deadly Gambits.'' The other is John Neuhaus, who's one of the most distinguished arms control specialists in our country. Both said that this administration turned down the "walk in the woods'' agreement first, and that would have been a perfect agreement from the standpoint of the United States in Europe and our security."

Please choose an answer form AA,BB,CC,DD or EE.
Before identifying the fallacy, explain your reasoning thoroughly. Your explanat